
# Kraken + ALTO OCR — Colab (Per‑Manuscript Projects, Best‑Model Saver)

This notebook keeps **everything per manuscript** inside a dedicated Drive folder and trains **recognition** models on ALTO data.

**What you get**
- One-time **Project ID** → creates `MyDrive/kraken_projects/<PROJECT_ID>/`
- Upload ALTO **ZIP** → auto-extracts to `data/`
- ALTO-first pairing → writes `train.txt` and `val.txt` to `lists/`
- **Controls** for: attempt number, learning rate, base model (previous/manual/auto-other)
- Training uses **`-t`/`-e`** (Kraken 3) and after training **copies the best checkpoint** to
  `models/rec/attempt_XX.mlmodel`
- CPU speed tweaks, lean Drive usage (small pip cache)


## 1) Connect Google Drive
This connects your Google Drive so the notebook can read/write your project files.
You’ll be asked to authorize once (click the link, choose your account, allow access).

Outcome: Drive mounted at /content/drive.

Tip: If you see a warning about “already mounted,” it’s fine.

In [ ]:

from google.colab import drive  # type: ignore
drive.mount('/content/drive')
print("✅ Drive mounted at /content/drive")


Mounted at /content/drive
✅ Drive mounted at /content/drive



## 2) Project Settings (edit only `PROJECT_ID`)

- `PROJECT_ID`: your manuscript ID (e.g., `0093`).  
This cell creates the project folder tree on Drive:
```
MyDrive/kraken_projects/<PROJECT_ID>/
├── data/          # upload & extracted dataset (ALTO XML + images)
├── lists/         # train.txt, val.txt
└── models/
    └── rec/       # attempt_01.mlmodel, attempt_02.mlmodel, ...
```

What to edit: only the PROJECT_ID (your manuscript number, e.g., 0093).
Outcome: Project folders created; paths printed.

In [ ]:

#@title 🔧 Project Settings
from pathlib import Path
import os

PROJECT_ID = "0093"  #@param {type:"string"}

ROOT_IN_DRIVE = "/content/drive/MyDrive"
PROJECTS_ROOT = f"{ROOT_IN_DRIVE}/kraken_projects"
PROJECT_DIR   = f"{PROJECTS_ROOT}/{PROJECT_ID}"
DATA_DIR      = f"{PROJECT_DIR}/data"
LISTS_DIR     = f"{PROJECT_DIR}/lists"
REC_MODELS    = f"{PROJECT_DIR}/models/rec"
PIP_CACHE_DIR = f"{ROOT_IN_DRIVE}/.pip-cache"   # small cache only

# Create the full project tree
for p in [PROJECTS_ROOT, PROJECT_DIR, DATA_DIR, LISTS_DIR, REC_MODELS, PIP_CACHE_DIR]:
    Path(p).mkdir(parents=True, exist_ok=True)

TRAIN_LIST = f"{LISTS_DIR}/train.txt"
VAL_LIST   = f"{LISTS_DIR}/val.txt"

# CPU basics
CORES = os.cpu_count() or 2
CPU_THREADS = max(2, CORES - 1)
DEVICE = "cpu"   # set "cuda" if you enable a T4 GPU in Colab

print("✅ Project folders ready")
print("PROJECT_DIR:", PROJECT_DIR)
print("DATA_DIR:", DATA_DIR)
print("LISTS_DIR:", LISTS_DIR)
print("REC_MODELS:", REC_MODELS)
print("CPU_THREADS:", CPU_THREADS, "| CORES:", CORES)


✅ Project folders ready
PROJECT_DIR: /content/drive/MyDrive/kraken_projects/0093
DATA_DIR: /content/drive/MyDrive/kraken_projects/0093/data
LISTS_DIR: /content/drive/MyDrive/kraken_projects/0093/lists
REC_MODELS: /content/drive/MyDrive/kraken_projects/0093/models/rec
CPU_THREADS: 2 | CORES: 2


## 3) CPU Speed Boost (threads & math libs)

Sets thread/env variables so image and math libraries don’t oversubscribe CPU cores.
Nothing to edit.
Outcome: Printed thread settings (this can shave seconds on CPU).

In [ ]:

import os
os.environ["OMP_NUM_THREADS"] = str(CPU_THREADS)
os.environ["MKL_NUM_THREADS"] = str(CPU_THREADS)
os.environ["OPENBLAS_NUM_THREADS"] = str(CPU_THREADS)
os.environ["NUMEXPR_NUM_THREADS"] = str(CPU_THREADS)
os.environ["KMP_BLOCKTIME"] = "1"
os.environ["KMP_SETTINGS"] = "0"
os.environ["KMP_AFFINITY"] = "granularity=fine,compact,1,0"

for k in ["OMP_NUM_THREADS","MKL_NUM_THREADS","OPENBLAS_NUM_THREADS","NUMEXPR_NUM_THREADS","KMP_BLOCKTIME","KMP_AFFINITY"]:
    print(k, "=", os.environ.get(k))


OMP_NUM_THREADS = 2
MKL_NUM_THREADS = 2
OPENBLAS_NUM_THREADS = 2
NUMEXPR_NUM_THREADS = 2
KMP_BLOCKTIME = 1
KMP_AFFINITY = granularity=fine,compact,1,0


## 4) Install Kraken (lean) + Pillow‑SIMD (faster image I/O)
Installs Kraken OCR and a faster Pillow build for image I/O.
The cell only installs if Kraken isn’t present.
Nothing to edit.
Outcome: “Kraken available/installed” and “Pillow-SIMD installed”.

In [ ]:

# Install Kraken (lean) + Pillow-SIMD (faster image IO)

import os, subprocess, sys

# Lean pip settings
os.environ["PIP_CACHE_DIR"] = PIP_CACHE_DIR
os.environ["PIP_DISABLE_PIP_VERSION_CHECK"] = "1"
os.environ["PIP_NO_INPUT"] = "1"

def is_importable(pkg: str) -> bool:
    try:
        __import__(pkg)
        return True
    except Exception:
        return False

# Install kraken only if missing
if is_importable("kraken"):
    import kraken
    print(f"✅ Kraken available (version: {getattr(kraken, '__version__', 'unknown')})")
else:
    print("⏳ Installing Kraken (lean) ...")
    subprocess.run([sys.executable, "-m", "pip", "-q", "install", "--upgrade", "pip"], check=True)
    subprocess.run([
        sys.executable, "-m", "pip", "-q", "install",
        "--prefer-binary",
        "--upgrade-strategy", "only-if-needed",
        "kraken[cairo]"
    ], check=True)
    import kraken, importlib
    importlib.reload(kraken)
    print(f"✅ Installed Kraken (version: {getattr(kraken, '__version__', 'unknown')})")

# Switch to Pillow-SIMD for faster image decoding/resizing
print("⏳ Switching to Pillow-SIMD for faster image ops...")
# Don't fail the whole cell if Pillow isn't present yet
subprocess.run([sys.executable, "-m", "pip", "-q", "uninstall", "-y", "pillow"], check=False)
subprocess.run([
    sys.executable, "-m", "pip", "-q", "install",
    "--prefer-binary",
    "--upgrade-strategy", "only-if-needed",
    "pillow-simd"
], check=True)
print("✅ Pillow-SIMD installed")


⏳ Installing Kraken (lean) ...
✅ Installed Kraken (version: unknown)
⏳ Switching to Pillow-SIMD for faster image ops...
✅ Pillow-SIMD installed


## 5) Upload your ALTO dataset (ZIP → Drive project folder)
Upload your ZIP containing images and ALTO XML files.
The ZIP is extracted into your project’s data/ folder on Drive.

What to do: Click the upload button and pick your ZIP.
Outcome: File listing of a few extracted files.

Your ZIP should contain paired page images (e.g., .tif/.jpg) and ALTO XMLs. The XMLs must include <fileName> elements referring to the page image.

In [ ]:
from google.colab import drive, files
from pathlib import Path
import zipfile

# Ensure Drive is mounted
if not Path("/content/drive/MyDrive").exists():
    drive.mount("/content/drive")

# Ensure PROJECT_ID exists (fallback is ok; better if set earlier)
if 'PROJECT_ID' not in globals():
    PROJECT_ID = "0093"  # fallback; user can change upstream

BASE_DIR = Path(f"/content/drive/MyDrive/kraken_projects/{PROJECT_ID}")
DATA_DIR = BASE_DIR / "data"
DATA_DIR.mkdir(parents=True, exist_ok=True)

print("📦 Please select your ZIP (ALTO XML + images)…")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("❌ No file uploaded.")

zip_name = next(iter(uploaded.keys()))
zip_path = Path(f"/content/{zip_name}")

print(f"✅ Uploaded: {zip_name}")
print(f"📂 Extracting into: {DATA_DIR}")

with zipfile.ZipFile(zip_path, "r") as zf:
    zf.extractall(DATA_DIR)

print("✅ Extraction done. Showing a few files:")
!find "{DATA_DIR}" -maxdepth 2 -type f | head -n 20


📦 Please select your ZIP (ALTO XML + images)…


Saving export_doc6047_4000_mirrored_alto_202512031528.zip to export_doc6047_4000_mirrored_alto_202512031528.zip
✅ Uploaded: export_doc6047_4000_mirrored_alto_202512031528.zip
📂 Extracting into: /content/drive/MyDrive/kraken_projects/0093/data
✅ Extraction done. Showing a few files:
/content/drive/MyDrive/kraken_projects/0093/data/Ave976_073v_mir.xml
/content/drive/MyDrive/kraken_projects/0093/data/Ave976_074r_mir.jpg
/content/drive/MyDrive/kraken_projects/0093/data/Ave976_072v_mir.jpg
/content/drive/MyDrive/kraken_projects/0093/data/Ave976_073r_mir.jpg
/content/drive/MyDrive/kraken_projects/0093/data/Ave976_075r_mir.jpg
/content/drive/MyDrive/kraken_projects/0093/data/Ave976_075v_mir.jpg
/content/drive/MyDrive/kraken_projects/0093/data/Ave976_074r_mir.xml
/content/drive/MyDrive/kraken_projects/0093/data/Ave976_073r_mir.xml
/content/drive/MyDrive/kraken_projects/0093/data/Ave976_075v_mir.xml
/content/drive/MyDrive/kraken_projects/0093/data/Ave976_072v_mir.xml
/content/drive/MyDrive/krak

## 6) Build train/val lists from ALTO (in project folder)
Scans data/ for ALTO XMLs and writes:
	•	lists/train.txt (90%)
	•	lists/val.txt (10%)

Important: For -f alto, lists must contain only XML file paths, one per line—no image paths.
Outcome: Counts printed + a short sample of each list.

If it says “Not enough ALTO XMLs” or shows missing files, check your ZIP structure.

In [ ]:
# Build train/val lists for ALTO: ONE XML PATH PER LINE (no image paths)

from pathlib import Path
import xml.etree.ElementTree as ET
import random, os

random.seed(42)  # reproducible split

DATA_DIR = DATA_DIR  # already defined
LISTS_DIR = LISTS_DIR
TRAIN_LIST = TRAIN_LIST
VAL_LIST = VAL_LIST

def is_alto_xml(p: Path) -> bool:
    if p.suffix.lower() != ".xml" or not p.is_file():
        return False
    try:
        # quick scan first
        with open(p, "r", encoding="utf-8", errors="ignore") as fh:
            head = fh.read(4096)
            if "<alto" not in head:
                return False
        # light parse to be safe
        ET.parse(p)
        return True
    except Exception:
        return False

# Collect all ALTO XMLs under DATA_DIR
xmls = [p for p in Path(DATA_DIR).rglob("*.xml") if is_alto_xml(p)]
xmls = sorted({x.resolve() for x in xmls})  # dedup + sort

n = len(xmls)
print(f"Found {n} ALTO XML files.")

if n < 2:
    raise SystemExit(f"❌ Not enough ALTO XMLs in {DATA_DIR}. Found {n}.")

# 90/10 split, shuffled
random.shuffle(xmls)
cut = max(1, int(n * 0.9))
train_xmls, val_xmls = xmls[:cut], xmls[cut:]

Path(LISTS_DIR).mkdir(parents=True, exist_ok=True)
with open(TRAIN_LIST, "w", encoding="utf-8") as f:
    for x in train_xmls:
        f.write(str(x) + "\n")
with open(VAL_LIST, "w", encoding="utf-8") as f:
    for x in val_xmls:
        f.write(str(x) + "\n")

print(f"✅ Wrote lists:")
print(f"  - {TRAIN_LIST}: {len(train_xmls)}")
print(f"  - {VAL_LIST}:   {len(val_xmls)}")

# quick sanity check
def show_sample(fp, k=5):
    print(f"\nSample from {fp}:")
    with open(fp, encoding="utf-8") as fh:
        for i, line in enumerate(fh):
            if i >= k: break
            print("  ", line.strip())
show_sample(TRAIN_LIST)
show_sample(VAL_LIST)

Found 7 ALTO XML files.
✅ Wrote lists:
  - /content/drive/MyDrive/kraken_projects/0093/lists/train.txt: 6
  - /content/drive/MyDrive/kraken_projects/0093/lists/val.txt:   1

Sample from /content/drive/MyDrive/kraken_projects/0093/lists/train.txt:
   /content/drive/MyDrive/kraken_projects/0093/data/Ave976_073r_mir.xml
   /content/drive/MyDrive/kraken_projects/0093/data/Ave976_074r_mir.xml
   /content/drive/MyDrive/kraken_projects/0093/data/Ave976_074v_mir.xml
   /content/drive/MyDrive/kraken_projects/0093/data/Ave976_073v_mir.xml
   /content/drive/MyDrive/kraken_projects/0093/data/Ave976_075v_mir.xml

Sample from /content/drive/MyDrive/kraken_projects/0093/lists/val.txt:
   /content/drive/MyDrive/kraken_projects/0093/data/Ave976_075r_mir.xml



## 7) Training Controls

Choose your attempt, learning rate, and base model source:
	•	BASE_SOURCE = choose → type the attempt number to load from this manuscript (e.g., 13).
	•	BASE_SOURCE = upload → you’ll upload a .mlmodel as base.

Inputs to fill:
	•	ATTEMPT_ID, LEARNING_RATE, BASE_SOURCE, and if needed BASE_ATTEMPT_TO_LOAD.
Outcome: The chosen values are printed for confirmation.

Tip: On CPU, try BATCH_SIZE = 32 (bump to 48/64 if it fits).


In [ ]:
#@title 🧭 Training Controls (choose or upload a base; no paths)
ATTEMPT_ID = 1            #@param {type:"number"}
LEARNING_RATE = 0.0001     #@param {type:"number"}  # set to 0 to use Kraken default
BASE_SOURCE = "upload"     #@param ["choose", "upload"]
BASE_ATTEMPT_TO_LOAD = None  #@param {type:"number"}  # attempt to load if BASE_SOURCE="choose"

# Early stopping
EARLY_STOP = "early"       #@param ["early", "dumb"]
LAG_EPOCHS = 6             #@param {type:"number"}
MIN_DELTA = 0.0001         #@param {type:"number"}

# CPU perf
import os
BATCH_SIZE = min(32, max(8, (os.cpu_count() or 2) * 2))

# Normalize LR (0 => None to use Kraken default)
LEARNING_RATE = None if (LEARNING_RATE == 0) else LEARNING_RATE

print("ATTEMPT_ID:", ATTEMPT_ID)
print("LEARNING_RATE:", LEARNING_RATE)
print("BASE_SOURCE:", BASE_SOURCE)
print("BASE_ATTEMPT_TO_LOAD:", BASE_ATTEMPT_TO_LOAD)

ATTEMPT_ID: 1
LEARNING_RATE: 0.0001
BASE_SOURCE: upload
BASE_ATTEMPT_TO_LOAD: None


## 🔗 Resolve Base Model (STRICT: choose attempt or upload; no fallback)
Picks the base .mlmodel automatically without asking for paths:
	•	If choose, it loads attempt_XX.mlmodel from this manuscript’s models/rec/.
	•	If upload, it opens a file picker and saves your upload to the right place.

Outcome: Prints the exact BASE_MODEL path. If a chosen attempt isn’t found, it lists available attempts.




In [ ]:
from pathlib import Path
from google.colab import files  # type: ignore
import shutil

# Clear any stale value
if 'BASE_MODEL' in globals():
    del BASE_MODEL

def list_available_attempts(rec_dir: str):
    recp = Path(rec_dir)
    found = sorted(p for p in recp.glob("attempt_*.mlmodel"))
    if not found:
        print("ℹ️ No attempts found yet in", recp)
        return
    print("Available attempts in this manuscript:")
    for p in found:
        print("  •", p.name)

def upload_model_to_drive(dest_dir: str) -> str:
    print("📤 Upload a .mlmodel file to use as base…")
    up = files.upload()
    if not up:
        raise SystemExit("❌ No file uploaded.")
    fname = next(iter(up.keys()))
    src = f"/content/{fname}"
    dest = str(Path(dest_dir) / Path(fname).name)
    shutil.move(src, dest)
    print(f"✅ Uploaded base to: {dest}")
    return dest

BASE_MODEL = None

if BASE_SOURCE == "choose":
    chosen = Path(REC_MODELS) / f"attempt_{int(BASE_ATTEMPT_TO_LOAD):02d}.mlmodel"
    if chosen.exists():
        BASE_MODEL = str(chosen.resolve())
        print("Base = chosen attempt:", BASE_MODEL)
    else:
        print(f"❌ attempt_{int(BASE_ATTEMPT_TO_LOAD):02d}.mlmodel not found in {REC_MODELS}.")
        list_available_attempts(REC_MODELS)
        raise SystemExit("Change BASE_ATTEMPT_TO_LOAD or switch to BASE_SOURCE='upload'.")

elif BASE_SOURCE == "upload":
    BASE_MODEL = upload_model_to_drive(REC_MODELS)

print("\n✅ BASE_MODEL resolved as:", BASE_MODEL)

📤 Upload a .mlmodel file to use as base…


Saving 0090_rec_03.mlmodel to 0090_rec_03.mlmodel
✅ Uploaded base to: /content/drive/MyDrive/kraken_projects/0093/models/rec/0090_rec_03.mlmodel

✅ BASE_MODEL resolved as: /content/drive/MyDrive/kraken_projects/0093/models/rec/0090_rec_03.mlmodel


## Stage ALTO dataset locally (XMLs + images; rewrite <fileName>)
Copies each XML in the lists and its linked image to local disk (/content/alto_staged/data/), then rewrites the XML’s <fileName> to the local image name. This removes Google Drive latency.

Nothing to edit.
Outcome: Creates:
	•	LOCAL_TRAIN_LIST and LOCAL_VAL_LIST with updated local XML paths
	•	A count of staged entries

On CPU, this is the biggest speed boost. On GPU, it’s optional (still helps a bit).


In [ ]:
#  Run staging for train/val (robust ALTO image resolution)


from pathlib import Path
import shutil
import re

# Expected inputs from earlier cells:
#   TRAIN_LIST, VAL_LIST
# Outputs used by training:
#   LOCAL_TRAIN_LIST, LOCAL_VAL_LIST, _USE_LOCAL
#
# If your notebook uses different variable names, adjust them here:
if 'TRAIN_LIST' not in globals():
    TRAIN_LIST = "/content/drive/MyDrive/kraken_projects/0093/lists/train.txt"
if 'VAL_LIST' not in globals():
    VAL_LIST = "/content/drive/MyDrive/kraken_projects/0093/lists/val.txt"

LOCAL_ROOT = Path("/content/alto_staged")
DATA_OUT = LOCAL_ROOT / "data"
LOCAL_TRAIN_LIST = LOCAL_ROOT / "train_local.txt"
LOCAL_VAL_LIST = LOCAL_ROOT / "val_local.txt"

LOCAL_ROOT.mkdir(parents=True, exist_ok=True)
DATA_OUT.mkdir(parents=True, exist_ok=True)

# Common export layouts (kept simple so users don't have to think)
IMAGE_SUBFOLDERS = ["", "images", "img", "pages", "page"]
IMAGE_EXTS = [".jpg", ".jpeg", ".png", ".tif", ".tiff"]

def _read_alto_filename(xml_text: str) -> str | None:
    """Extracts <fileName>...</fileName> from ALTO XML."""
    m = re.search(r"<fileName>\s*([^<]+)\s*</fileName>", xml_text)
    return m.group(1).strip() if m else None

def _find_image_for_alto(xml_path: Path, file_name: str) -> Path | None:
    """Finds the image file referenced by ALTO, searching common folders and extension/case variants."""
    # 1) Try exact file name in common subfolders
    for sub in IMAGE_SUBFOLDERS:
        candidate = xml_path.parent / sub / file_name
        if candidate.exists():
            return candidate

    # 2) Try stem + extension variants (and upper-case extensions)
    stem = Path(file_name).stem
    for sub in IMAGE_SUBFOLDERS:
        base = xml_path.parent / sub
        for ext in IMAGE_EXTS:
            for variant in (ext, ext.upper()):
                candidate = base / (stem + variant)
                if candidate.exists():
                    return candidate

    return None

def _stage_list(list_path: str, out_list_path: Path, data_out: Path) -> int:
    """
    Stages ALTO XMLs + referenced images into `data_out` and writes a local list file.
    Returns number of staged entries.
    """
    list_path = Path(list_path)
    out_list_path.parent.mkdir(parents=True, exist_ok=True)
    staged = 0
    missing = []

    with list_path.open("r", encoding="utf-8") as f_in, out_list_path.open("w", encoding="utf-8") as f_out:
        for raw in f_in:
            raw = raw.strip()
            if not raw:
                continue

            src_xml = Path(raw)
            if not src_xml.exists():
                missing.append(f"Missing XML: {src_xml}")
                continue

            xml_text = src_xml.read_text(encoding="utf-8", errors="ignore")
            file_name = _read_alto_filename(xml_text)

            if not file_name:
                missing.append(f"No <fileName> in: {src_xml}")
                continue

            img_path = _find_image_for_alto(src_xml, file_name)
            if img_path is None:
                missing.append(f"Image not found for: {src_xml} (<fileName>={file_name})")
                continue

            # Copy XML into local staging
            dst_xml = data_out / src_xml.name
            if not dst_xml.exists():
                shutil.copy2(src_xml, dst_xml)

            # Copy image into local staging under the exact <fileName> expected by ALTO
            dst_img = data_out / file_name
            if not dst_img.exists():
                shutil.copy2(img_path, dst_img)

            # Write staged XML path into local list (Kraken reads image via ALTO reference)
            f_out.write(str(dst_xml) + "\n")
            staged += 1

    print(f"✅ Staged {staged} entries → {out_list_path}")
    if missing:
        print(f"⚠️ Skipped {len(missing)} items (showing up to 5):")
        for m in missing[:5]:
            print("  -", m)

    # Fail early if empty so users aren't confused later
    if staged == 0:
        raise RuntimeError(
            "Staging produced 0 entries.\n"
            "This usually means your ALTO XML references images that are not available next to the XML\n"
            "or inside a common subfolder (images/img/pages/page), OR your export did not include images.\n"
            "Fix: ensure the export includes the page images, or place them into one of those folders."
        )

    return staged

# Run staging for train/val
n_tr = _stage_list(TRAIN_LIST, LOCAL_TRAIN_LIST, DATA_OUT)
n_va = _stage_list(VAL_LIST, LOCAL_VAL_LIST, DATA_OUT)

print("\nLocal staged dataset:")
print("  Root:", LOCAL_ROOT)
print("  Data dir:", DATA_OUT)
print("  TRAIN (local):", LOCAL_TRAIN_LIST, "| entries:", n_tr)
print("  VAL   (local):", LOCAL_VAL_LIST,   "| entries:", n_va)

# Tell training cell to use staged lists
_USE_LOCAL = True
print("\n👉 In your training cell set: _USE_LOCAL = True (or keep it enabled if already true).")


✅ Staged 6 entries → /content/alto_staged/train_local.txt
✅ Staged 1 entries → /content/alto_staged/val_local.txt

Local staged dataset:
  Root: /content/alto_staged
  Data dir: /content/alto_staged/data
  TRAIN (local): /content/alto_staged/train_local.txt | entries: 6
  VAL   (local): /content/alto_staged/val_local.txt | entries: 1

👉 In your training cell set: _USE_LOCAL = True (or keep it enabled if already true).


[link text](https://)
## 8) Train recognition (best checkpoint → `attempt_XX.mlmodel`)

Trains with live logs (you’ll see epochs and metrics as they happen).
The best checkpoint is saved as:
MyDrive/kraken_projects/<PROJECT_ID>/models/rec/attempt_XX.mlmodel

What to check before running:
	•	BASE_MODEL is printed correctly (from “Resolve Base Model”).
	•	_USE_LOCAL = True if you ran “Stage ALTO” (recommended on CPU).

Outcome: Live training logs; at the end you’ll see “Saved best checkpoint as …”.

Slow on CPU? Use staging, increase BATCH_SIZE, validate less often (set -F 2), or switch Colab runtime to GPU (T4).

In [ ]:
import os
import subprocess
import shlex
import textwrap

# Try to infer PROJECT_ID and BASE_DIR if they are not already defined


if 'PROJECT_ID' not in globals():
    if 'BASE_MODEL' in globals() and BASE_MODEL:
        parts = BASE_MODEL.split('/')
        if "kraken_projects" in parts:
            idx = parts.index("kraken_projects")
            if idx + 1 < len(parts):
                PROJECT_ID = parts[idx + 1]
            else:
                PROJECT_ID = "default_project"
        else:
            PROJECT_ID = "default_project"
    else:
        PROJECT_ID = "0093"  # fallback, change if needed

if 'BASE_DIR' not in globals():
    BASE_DIR = f"/content/drive/MyDrive/kraken_projects/{PROJECT_ID}"

# ---------------------------------------------------------------------------
# Train/val lists – use existing vars if defined, otherwise defaults
# ---------------------------------------------------------------------------

_TR = TRAIN_LIST if 'TRAIN_LIST' in globals() else "/content/alto_staged/train_local.txt"
_VA = VAL_LIST   if 'VAL_LIST'   in globals() else "/content/alto_staged/val_local.txt"

# ---------------------------------------------------------------------------
# Output directory for this recognizer attempt
# ---------------------------------------------------------------------------

REC_MODELS_DIR = os.path.join(BASE_DIR, "models", "rec")
os.makedirs(REC_MODELS_DIR, exist_ok=True)

# ATTEMPT_ID normalization
if 'ATTEMPT_ID' not in globals():
    ATTEMPT_ID = "attempt_01"
else:
    if isinstance(ATTEMPT_ID, int):
        ATTEMPT_ID = f"attempt_{ATTEMPT_ID:02d}"
    else:
        ATTEMPT_ID = str(ATTEMPT_ID)

out_prefix = os.path.join(REC_MODELS_DIR, ATTEMPT_ID)

print("📁 Project ID:     ", PROJECT_ID)
print("📂 BASE_DIR:       ", BASE_DIR)
print("📄 Train list:     ", _TR)
print("📄 Val list:       ", _VA)
print("📦 Output prefix:  ", out_prefix)
print("🧠 Base model:     ", BASE_MODEL if 'BASE_MODEL' in globals() else None)
print()

# ---------------------------------------------------------------------------
# Build ketos train command
# ---------------------------------------------------------------------------

if 'LAG_EPOCHS' not in globals():
    LAG_EPOCHS = 6
if 'MIN_DELTA' not in globals():
    MIN_DELTA = 0.0001
if 'BATCH_SIZE' not in globals():
    BATCH_SIZE = 8
# LEARNING_RATE may be None → kraken default

cmd = [
    "ketos", "train",
    "-o", out_prefix,
    "-f", "alto",
    "-t", _TR,
    "-e", _VA,
    "-q", "early",
    "--lag", str(int(LAG_EPOCHS)),
    "--min-delta", str(float(MIN_DELTA)),
    "-B", str(int(BATCH_SIZE)),
    "-F", "1",
]

if 'BASE_MODEL' in globals() and BASE_MODEL:
    cmd += ["-i", BASE_MODEL, "--resize", "add"]

if 'LEARNING_RATE' in globals() and LEARNING_RATE is not None:
    cmd += ["-r", str(LEARNING_RATE)]

print("Running:")
print(textwrap.fill(" ".join(shlex.quote(c) for c in cmd), width=120))
print("-" * 80)

# ---------------------------------------------------------------------------
# Run training (interrupt-aware, line-by-line streaming)
# ---------------------------------------------------------------------------

BEST_MODEL_PATH = None
proc = None

try:
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
    )

    for line in proc.stdout:
        print(line, end="")

    proc.stdout.close()
    proc.wait()

except KeyboardInterrupt:
    print("\n⛔ Training interrupted by user. Trying to stop ketos…")
    if proc is not None and proc.poll() is None:
        proc.terminate()
        try:
            proc.wait(timeout=10)
        except Exception:
            pass

finally:
    best_candidate = out_prefix + "_best.mlmodel"
    if os.path.exists(best_candidate):
        BEST_MODEL_PATH = best_candidate
        print(f"\n🌟 Best model found at: {BEST_MODEL_PATH}")
    else:
        print("\n⚠️ No _best.mlmodel found yet (did training reach at least one epoch?)")

    if proc is not None and proc.returncode not in (0, None):
        print(f"⚠️ ketos exited with return code {proc.returncode}.")


📁 Project ID:      0093
📂 BASE_DIR:        /content/drive/MyDrive/kraken_projects/0093
📄 Train list:      /content/drive/MyDrive/kraken_projects/0093/lists/train.txt
📄 Val list:        /content/drive/MyDrive/kraken_projects/0093/lists/val.txt
📦 Output prefix:   /content/drive/MyDrive/kraken_projects/0093/models/rec/attempt_01
🧠 Base model:      /content/drive/MyDrive/kraken_projects/0093/models/rec/0090_rec_03.mlmodel

Running:
ketos train -o /content/drive/MyDrive/kraken_projects/0093/models/rec/attempt_01 -f alto -t
/content/drive/MyDrive/kraken_projects/0093/lists/train.txt -e /content/drive/MyDrive/kraken_projects/0093/lists/val.txt
-q early --lag 6 --min-delta 0.0001 -B 8 -F 1 -i
/content/drive/MyDrive/kraken_projects/0093/models/rec/0090_rec_03.mlmodel --resize add -r 0.0001
--------------------------------------------------------------------------------
2025-12-04 14:25:01.368787: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: At

## 9) Download the best model

> Add blockquote



In [ ]:
import os
from google.colab import files
from ipywidgets import Dropdown, Button, VBox, Output
from IPython.display import display

# --- Adjust this if your project path pattern ever changes ---
if 'PROJECT_ID' not in globals():
    PROJECT_ID = "0093"  # fallback; change if needed

BASE_DIR = f"/content/drive/MyDrive/kraken_projects/{PROJECT_ID}"
REC_MODELS_DIR = os.path.join(BASE_DIR, "models", "rec")

if not os.path.isdir(REC_MODELS_DIR):
    raise RuntimeError(f"Recognizer models directory not found: {REC_MODELS_DIR}")

# Find all .mlmodel files in the recognizer directory
all_models = [
    f for f in os.listdir(REC_MODELS_DIR)
    if f.endswith(".mlmodel")
]

if not all_models:
    raise RuntimeError("No .mlmodel files found yet. Train at least one model first.")

# Prefer *_best.mlmodel if they exist; otherwise fall back to all models
best_models = [f for f in all_models if f.endswith("_best.mlmodel")]

if best_models:
    print("Found best models:")
    model_files = sorted(best_models)
else:
    print("⚠️ No *_best.mlmodel files found; listing all models instead.")
    model_files = sorted(all_models)

# Build dropdown options: label → full path
options = []
for f in model_files:
    label = f.replace(".mlmodel", "")  # e.g. "attempt_01_best"
    full_path = os.path.join(REC_MODELS_DIR, f)
    options.append((label, full_path))

attempt_dropdown = Dropdown(
    options=options,
    description="Model:",
    disabled=False,
)

download_button = Button(
    description="Download selected model",
    button_style="",  # "success", "info", etc. if you like
)

out = Output()

def on_download_clicked(b):
    with out:
        out.clear_output()
        model_path = attempt_dropdown.value
        if os.path.exists(model_path):
            print(f"Downloading: {model_path}")
            files.download(model_path)
        else:
            print(f"❌ File not found: {model_path}")

download_button.on_click(on_download_clicked)

display(VBox([attempt_dropdown, download_button, out]))



Found best models:


## 9) Evaluate (CER/WER)

In [ ]:

import shlex, subprocess

cmd = ["ketos", "test", "-f", "alto", "-m", final_best, VAL_LIST]
print("Running:", " ".join(shlex.quote(x) for x in cmd))
res = subprocess.run(cmd, text=True)

if res.returncode == 0:
    print("✅ Evaluation completed. See metrics above.")
else:
    raise SystemExit("❌ Evaluation failed. Check the logs above.")


NameError: name 'final_best' is not defined


### Notes
- Change only `PROJECT_ID` in Project Settings; keep **everything else** under the project folder.
- For each new attempt, adjust **ATTEMPT_ID**, **LEARNING_RATE**, and **BASE_SOURCE** in the **Training Controls** cell.
- The best checkpoint for each attempt is copied to `attempt_XX.mlmodel` so you can chain attempts cleanly.
- To warm-start a new manuscript from another one, keep **BASE_SOURCE = "auto_other"** for attempt 01 (or point to a model with `"manual"`).
- For much faster training, switch to a **T4 GPU** in Colab and set `DEVICE = "cuda"`.
